# Analyze Data

In [ ]:
library(arrow, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(ggplot2)
library(scales)
library(survival)
library(survminer)
library(splines)
library(emmeans)
library(stringr)
library(xtable)

In [ ]:
library(truveta.research)

In [ ]:
source(here::here("wrangle_scripts", "R", "_.R"))

In [ ]:
results_dir <- here::here("results")
data_dir <- here::here("data")
dir.create(data_dir, recursive = TRUE, showWarnings = FALSE)

In [ ]:
conditions <- c('ckd', 'diabetes', 'immunocompromised', 'lung')

In [ ]:
initialize_theme_truveta(figsize = c(10, 7))

In [ ]:
make_summary_graph <- function(summary_table, ncol = 2) {
  out <- 
    summary_table |>
    dplyr::filter(term %in% conditions) |>
    ggplot(aes(x = estimate, y = term)) +
    geom_vline(xintercept = 1, linetype = 'dashed') +
    geom_pointrange(mapping = aes(xmin = conf_low, xmax = conf_high)) +
    facet_wrap(~ model, ncol = ncol) +
    theme_truveta()
    
  out
}


In [ ]:
clean_multi_vs_base <- function(emmeans_pairs, conditions, select, column) {
  temp <-
    emmeans_pairs |>
    confint() |>
    janitor::clean_names() |>
    tidyr::separate(
      contrast, 
      into = c('numerator', 'denominator'), 
      sep = ' / '
    ) |>
    dplyr::filter(denominator == '0 0 0 0') |> 
    tidyr::separate(
      numerator, 
      into = conditions, 
      sep = ' '
    ) |>
    dplyr::mutate(
      dplyr::across(
        lung:ckd, 
        ~ if_else(.x == 1, deparse(substitute(.x)), NA_character_)
      )
    ) |>
    tidyr::unite(col = 'condition', {{ select }}, sep = ', ', na.rm = TRUE) |>
    dplyr::select(condition, {{ column }}, asymp_lcl, asymp_ucl)
    
  temp
}

## Load Data

In [ ]:
df <- read_parquet_table(file.path(data_dir, "feature_table.parquet"), results_dir)
#head(df)

In [ ]:
df <- 
  df |>
  dplyr::rename(
    age_years = vaccination_years,
    age_bracket = vaccination_bracket,
    month_year = monitoring_date
  ) |>
  dplyr::mutate(month_year = as.character(month_year))

## Breakthrough infection

### Survival analysis

In [ ]:
df_surv <- 
  df |>
  dplyr::select(
    outcome_time, 
    covid, 
    ckd, 
    diabetes, 
    immunocompromised, 
    lung, 
    race, 
    ethnicity, 
    sex, 
    age_years,
    #age_bracket,
    month_year
  )

In [ ]:
# four way interactions
fit_surv_four <- 
  coxph(
    Surv(time = outcome_time, event = covid) ~ 
      (ckd + diabetes + immunocompromised + lung)^4 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_surv
  )

In [ ]:
# three way interactions
fit_surv_three <- 
  coxph(
    Surv(time = outcome_time, event = covid) ~ 
      (ckd + diabetes + immunocompromised + lung)^3 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_surv
  )

In [ ]:
# two way interactions
fit_surv_two <- 
  coxph(
    Surv(time = outcome_time, event = covid) ~ 
      (ckd + diabetes + immunocompromised + lung)^2 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_surv
  )

In [ ]:
# no interactions
fit_surv_base <-
  coxph(
    Surv(time = outcome_time, event = covid) ~ 
      ns(age_years, df = 5) + . - age_years,
    data = df_surv
  )

#### summarize models

In [ ]:
model_list <- 
  list(
    four_way = fit_surv_four, 
    three_way = fit_surv_three, 
    two_way = fit_surv_two, 
    base = fit_surv_base
  )

In [ ]:
summary_table <- make_model_summary(model_list)

write_table(summary_table, file.path(results_dir, "breakthrough_survival_summary_table.csv"))

#summary_table

In [ ]:
break_survival_summary <- 
  make_summary_graph(summary_table) +
  labs(
    title = str_wrap('Hazard Ratio of Breakthrough COVID infection associated with comorbidities', 50), 
    subtitle = 'presented with 95% confidence intervals',
    x = 'Hazard Ratio', 
    y = 'Comorbidity'
  )

write_ggplot(
  break_survival_summary, 
  file.path(results_dir, "breakthrough_survival_summary.png")
)

break_survival_summary

In [ ]:
tab_aic <- make_aic_table(model_list)

tab_aic_c <- make_aic_table(model_list, aic_c, name = 'AICc')

aic_table <- 
  dplyr::full_join(tab_aic, tab_aic_c, by = 'model') |>
  dplyr::mutate(
    model = stringr::str_to_title(model),
    model = stringr::str_replace(model, '_', '-'),
    across(AIC:delta_AICc, ~ round(.x, 2))
  ) |>
  dplyr::rename(
    `Model complexity` = model,
    `\\delta AIC` = delta_AIC,
    `\\delta AICc` = delta_AICc
  )

write_table(aic_table, file.path(results_dir, "breakthrough_survival_aic_table.csv"))

caption <- 'Comparison between four candidate models of time till breakthrough COVID-19 infection each with varying degrees of interaction between comorbidities.'
label <- 'tab:breakthrough_aic'


xtable::print.xtable(
  xtable::xtable(
    as.data.frame(aic_table, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'breakthrough_survival_aic_table.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

aic_table

In [ ]:
# summarize best model

summary_fit <- broom::tidy(fit_surv_three, exponentiate = TRUE, conf.int = TRUE)

In [ ]:
# write summary

write_table(summary_fit, file = file.path(results_dir, 'model_summary_survival_breakthrough.csv'))

caption <- 
  paste0(
    'Summary of the selected Cox proportaional hazards model of time till breakthrough infection ',
    'showing estimates for all of the regression coefficients. Estimates are on the hazard scale (i.e. exponentiated).'
  )
label <- 'tab:summary_survival_breakthrough'

xtable::print.xtable(
  xtable::xtable(
    as.data.frame(summary_fit, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'model_summary_survival_breakthrough.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

In [ ]:
# multi way model effects
em <- emmeans(fit_surv_three, ~ (ckd + diabetes + immunocompromised + lung)^3, type = 'response')

pp <- pairs(em, reverse = TRUE)

surv_multi_hr <- 
  clean_multi_vs_base(pp, conditions, ckd:lung, ratio) |>
  dplyr::mutate(
    condition = str_replace_all(condition, 'lung', 'cld'),
    condition = str_replace_all(condition, 'immunocompromised', 'immuno.'),
    condition = str_replace_all(condition, 'diabetes', 'diab.'),
    condition = 
      case_when(
        condition == 'ckd, cld' ~ 'ckd, cld',
        condition == 'diab., cld' ~ 'cld, diab.',
        condition == 'immuno., cld' ~ 'cld, immuno.',
        condition == 'ckd, diab., cld' ~ 'ckd, cld, diab.',
        condition == 'ckd, immuno., cld' ~ 'ckd, cld, immuno.',
        condition == 'diab., immuno., cld' ~ 'cld, diab., immuno.',
        condition == 'ckd, diab., immuno., cld' ~ 'ckd, cld, diab., immuno.',
        TRUE ~ condition
      ),
    condition = 
      factor(
        condition, 
        levels = 
          c(
            'ckd', 
            'cld', 
            'diab.', 
            'immuno.', 
            'ckd, cld', 
            'ckd, diab.', 
            'ckd, immuno.', 
            'cld, diab.', 
            'cld, immuno.',
            'diab., immuno.',
            'ckd, cld, diab.',
            'ckd, diab., immuno.',
            'ckd, cld, immuno.',
            'cld, diab., immuno.',
            'ckd, cld, diab., immuno.'
          )
      )
  )


In [ ]:
temp <- 
  surv_multi_hr |>
  dplyr::mutate(
    dplyr::across(ratio:asymp_ucl, ~ round(.x, 2)),
    or = paste0(ratio, ' [', asymp_lcl, ', ', asymp_ucl, ']')
  ) |> 
  dplyr::select(condition, or) |>
  dplyr::arrange(condition)

colnames(temp) <- c('Comorbidities', 'Hazard Ratio [95% CI]')

temp

write_table(temp, file = file.path(results_dir, 'survival_hazard_breakthrough.csv'))

caption <- 
  paste0(
     'Estimated hazard ratios of breakthrough infection associated patients having one ',
    'or more of the studied comorbidities copmared to patients who have none of the studied comorbidities. ',
    'CKD: chronic kidney disease, CLD: chronic lung disease, Diab.: diabetes, Immuno.: immunocompromised.'
  )
label <- 'tab:survival_hazard_breakthrough'

xtable::print.xtable(
  xtable::xtable(
    as.data.frame(temp, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'survival_hazard_breakthrough.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

In [ ]:
surv_multi_hr_gg <- 
  surv_multi_hr |>
  ggplot(aes(x = ratio, y = condition)) +
  geom_vline(xintercept = 1, linetype = 'dashed') +
  geom_pointrange(mapping = aes(xmin = asymp_lcl, xmax = asymp_ucl), size = 1.15) +
  theme_truveta() +
  labs(
    title = str_wrap('Hazard ratio of breakthrough COVID-19 infection associated with a combination of comorbidities', 50),
    subtitle = 'presented with 95% confidence intervals',
    x = 'Hazard Ratio', 
    y = 'Comorbdities in combination'
  )

write_ggplot(
  surv_multi_hr_gg, 
  file.path(results_dir, "breakthrough_survival_hazard_interactions.png")
)

surv_multi_hr_gg

In [ ]:
surv_multi_hr_gg_flip <- 
  surv_multi_hr_gg + 
  coord_flip() +
  theme(
    text = element_text(family = 'Open Sans'),
    plot.title = element_text(size = 20),
    axis.title = element_text(size = 17.5),
    axis.title.x = element_text(hjust = 0),
    axis.text.y = element_text(size = 14),
    axis.text.x = element_text(size = 14, colour = 'black', angle = 45, vjust = 1, hjust = 1),
    plot.caption = element_text(hjust = 0)
  ) +
  labs(
    title = str_wrap('Hazard ratio of breakthrough COVID-19 infection associated with a combination of comorbidities', 55),
    caption = glue::glue(
        "ckd  ........... chronic kidney disease\n",
        "cld  ............ chronic lung disease\n",
        "diab.  ......... diabetes\n",
        "immuno.  ... immunocompromised",
    )
  ) +
  scale_x_continuous(breaks = c(1, 3, 6, 9)) 

# grid::grid.text("xxx", x = unit(0.91, "npc"), y = unit(0.80, "npc"))

write_ggplot(
  surv_multi_hr_gg_flip, 
  file.path(results_dir, "breakthrough_survival_hazard_interactions_flip.png"),
  width = 10,
  height = 7
)

surv_multi_hr_gg_flip

## Hospitalization following breakthrough infection

### Logistic regression

In [ ]:
df_hospital <- 
  df |> 
  dplyr::filter(covid == 1)

In [ ]:
df_hospital_logistic <- 
  df_hospital |>
  dplyr::select(
    hospital, 
    ckd, 
    diabetes, 
    immunocompromised, 
    lung, 
    race, 
    ethnicity, 
    sex, 
    age_years,
    #age_bracket,
    month_year
  )

In [ ]:
# four way interactions
fit_logi_four <- 
  glm(
    hospital ~ 
      (ckd + diabetes + immunocompromised + lung)^4 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_hospital_logistic,
    family = "binomial"
  )

In [ ]:
# three way interactions
fit_logi_three <- 
  glm(
    hospital ~ 
      (ckd + diabetes + immunocompromised + lung)^3 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_hospital_logistic,
    family = "binomial"
  )

In [ ]:
# two way interactions
fit_logi_two <- 
  glm(
    hospital ~ 
      (ckd + diabetes + immunocompromised + lung)^2 + 
      ns(age_years, df = 5) + . - age_years,
    data = df_hospital_logistic,
    family = "binomial"
  )

In [ ]:
# two way interactions
fit_logi_base <- 
  glm(
    hospital ~ ns(age_years, df = 5) + . - age_years,
    data = df_hospital_logistic,
    family = "binomial"
  )

#### summarize models

In [ ]:
model_list <- 
  list(
    four_way = fit_logi_four,
    three_way = fit_logi_three,
    two_way = fit_logi_two, 
    base = fit_logi_base
  )

In [ ]:
summary_table <- make_model_summary(model_list)

write_table(summary_table, file.path(results_dir, "hospital_logistic_summary_table.csv"))

#summary_table

In [ ]:
hospital_logistic_summary <- 
  make_summary_graph(summary_table) +
  labs(
    title = str_wrap('Odds ratio of hospitalization following breakthrough COVID infection associated with comorbidities', 40),
    subtitle = 'presented with 95% confidence intervals',
    x = 'Odds Ratio', 
    y = 'Comorbidity'
  )

write_ggplot(
  hospital_logistic_summary, 
  file.path(results_dir, "hospital_logistic_summary.png")
)

hospital_logistic_summary

In [ ]:
tab_aic <- make_aic_table(model_list)

tab_aic_c <- make_aic_table(model_list, aic_c, name = 'AICc')

aic_table <- 
  dplyr::full_join(tab_aic, tab_aic_c, by = 'model') |>
  dplyr::mutate(
    model = stringr::str_to_title(model),
    model = stringr::str_replace(model, '_', '-'),
    across(AIC:delta_AICc, ~ round(.x, 2))
  ) |>
  dplyr::rename(
    `Model complexity` = model,
    `\\delta AIC` = delta_AIC,
    `\\delta AICc` = delta_AICc
  )

write_table(aic_table, file.path(results_dir, "hospital_logistic_aic_table.csv"))


caption <- 
  paste0(
    'Comparison between four candidate models of probability of hospitalization following ',
    'breakthrough COVID-19 infection each with varying degrees of interaction between comorbidities.'
  )
label <- 'tab:hospital_aic'


xtable::print.xtable(
  xtable::xtable(
    as.data.frame(aic_table, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'hospital_logistic_aic_table.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

aic_table

In [ ]:
# summarize "best" model

summary_fit <- broom::tidy(fit_logi_base, exponentiate = TRUE, conf.int = TRUE)

In [ ]:
# write summary

write_table(summary_fit, file = file.path(results_dir, 'model_summary_logistic_hospital.csv'))

caption <- 
  paste0(
    'Summary of the selected logistic regression model of hospitalization following breakthrough ',
    'infection showing estimates for all of the regression coefficients. Estimates are on the odds scale (i.e. exponentiated).'
  )
label <- 'tab:summary_logistic_hospital'

xtable::print.xtable(
  xtable::xtable(
    as.data.frame(summary_fit, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'model_summary_logistic_hospital.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

In [ ]:
# multi way model effects
em <- emmeans(fit_logi_base, ~ (ckd + diabetes + immunocompromised + lung), type = 'response')

pp <- pairs(em, reverse = TRUE)

In [ ]:
logi_multi_or <- 
  clean_multi_vs_base(pp, conditions, ckd:lung, odds_ratio) |>
  dplyr::mutate(
    condition = str_replace_all(condition, 'lung', 'cld'),
    condition = str_replace_all(condition, 'immunocompromised', 'immuno.'),
    condition = str_replace_all(condition, 'diabetes', 'diab.'),
    condition = 
      case_when(
        condition == 'ckd, cld' ~ 'ckd, cld',
        condition == 'diab., cld' ~ 'cld, diab.',
        condition == 'immuno., cld' ~ 'cld, immuno.',
        condition == 'ckd, diab., cld' ~ 'ckd, cld, diab.',
        condition == 'ckd, immuno., cld' ~ 'ckd, cld, immuno.',
        condition == 'diab., immuno., cld' ~ 'cld, diab., immuno.',
        condition == 'ckd, diab., immuno., cld' ~ 'ckd, cld, diab., immuno.',
        TRUE ~ condition
      ),
    condition = 
      factor(
        condition, 
        levels = 
          c(
            'ckd', 
            'cld', 
            'diab.', 
            'immuno.', 
            'ckd, cld', 
            'ckd, diab.', 
            'ckd, immuno.', 
            'cld, diab.', 
            'cld, immuno.',
            'diab., immuno.',
            'ckd, cld, diab.',
            'ckd, diab., immuno.',
            'ckd, cld, immuno.',
            'cld, diab., immuno.',
            'ckd, cld, diab., immuno.'
          )
      )
  )

In [ ]:
head(logi_multi_or)

In [ ]:
temp <- 
  logi_multi_or |>
  dplyr::mutate(
    dplyr::across(odds_ratio:asymp_ucl, ~ round(.x, 2)),
    or = paste0(odds_ratio, ' [', asymp_lcl, ', ', asymp_ucl, ']')
  ) |> 
  dplyr::select(condition, or) |>
  dplyr::arrange(condition)

colnames(temp) <- c('Comorbidities', 'Odds Ratio [95% CI]')

temp

write_table(temp, file = file.path(results_dir, 'logistic_odds_hospital.csv'))

caption <- 
  paste0(
    'Estimated odds ratios of hospitalization associated patients having one or more of the studied ',
    'comorbidities copmared to patients who have none of the studied comorbidities. ',
    'CKD: chronic kidney disease, CLD: chronic lung disease, Diab.: diabetes, Immuno.: immunocompromised.'
  )
label <- 'tab:logistic_odds_hospital'

xtable::print.xtable(
  xtable::xtable(
    as.data.frame(temp, make.names = FALSE), 
    label = label,
    caption = caption
  ),
  type = 'latex', 
  file = here::here('results', 'logistic_odds_hospital.tex'), 
  include.rownames = FALSE,
  comment = FALSE,
  timestamp = NULL,
  table.placement = '!htbp'
)

In [ ]:
logi_multi_or_gg <- 
  logi_multi_or |>
  ggplot(aes(x = odds_ratio, y = condition)) +
  geom_vline(xintercept = 1, linetype = 'dashed') +
  geom_pointrange(mapping = aes(xmin = asymp_lcl, xmax = asymp_ucl), size = 1.15) +
  theme_truveta() +
  labs(
    title = str_wrap('Odds ratio of hospitalization following COVID-19 infection associated with a combination of comorbidities', 50),
    subtitle = 'presented with 95% confidence intervals',
    x = 'Odds Ratio', 
    y = 'Comorbdities in combination'
  )

write_ggplot(
  logi_multi_or_gg, 
  file.path(results_dir, "hospital_logistic_odds_interactions.png")
)

logi_multi_or_gg

In [ ]:
logi_multi_or_gg_flip <-
  logi_multi_or_gg + 
  coord_flip() +
  theme(
    text = element_text(family = 'Open Sans'),
    plot.title = element_text(size = 20),
    axis.title = element_text(size = 17.5),
    axis.title.x = element_text(hjust = 0),
    axis.text.y = element_text(size = 14),
    axis.text.x = element_text(size = 14, colour = 'black', angle = 45, vjust = 1, hjust = 1),
    plot.caption = element_text(hjust = 0)
  ) +
  labs(
    title = str_wrap('Odds ratio of hospitalization following COVID-19 infection associated with a combination of comorbidities', 65),
    caption = glue::glue(
        "ckd  ........... chronic kidney disease\n",
        "cld  ............ chronic lung disease\n",
        "diab.  ......... diabetes\n",
        "immuno.  ... immunocompromised",
    )
  ) +
  scale_x_continuous(breaks = c(1, 3, 6, 9, 12))

write_ggplot(
  logi_multi_or_gg_flip, 
  file.path(results_dir, "hospital_logistic_odds_interactions_flip.png"),
  width = 10,
  height = 7
)

logi_multi_or_gg_flip